# Plan

- testing theory overview:
    - why: check that your code is valid
    - unit tests: check only specific code (module, func), "mock" everyting else (use some stubs intead of real foreign functions)
    - type, features: mock, fixtures, expect exceptions
- doctest: the simplest way, + part of documentation (example of usage)
- pytest (include pdb, benchmarking), unittest
    - see some github repo (any big enough repo has tests =)) as an example
    - see https://docs.python.org/3/library/pdb.html how to work with pdb
- nose (simpler, than prevs, can be run even without any imports), tox

## Docs

http://docs.python-guide.org/en/latest/writing/tests/

### pytest

http://pythontesting.net/start-here/
https://docs.pytest.org/en/latest/
https://semaphoreci.com/community/tutorials/testing-python-applications-with-pytest/

### unittest

http://pythontesting.net/framework/unittest/unittest-introduction/
https://docs.python.org/3/library/unittest.html

### nosetest

http://pythontesting.net/framework/nose/nose-introduction/

can run even very simpler functions without any extra stuff like:
```python
def test_numbers_3_4():
    assert multiply(3,4) == 12 
```

### tox

https://tox.readthedocs.io/en/latest/

# Doctest

In [4]:
# try to set a wrong expected value
# def fun():
#     """
#     >>> fun()
#     0
#     """
#     return 42

def fun():
    """
    >>> fun()
    Traceback (most recent call last):
      File "/home/haskfp/tools/miniconda3/lib/python3.6/doctest.py", line 1330, in __run
        compileflags, 1), test.globs)
      File "<doctest __main__.fun[0]>", line 1, in <module>
        fun()
      File "<ipython-input-3-c8c65832d300>", line 14, in fun
        return 42/0
    ZeroDivisionError: division by zero
    """
    return 42/0

def fun_with_arg(a, b=0):
    """
    >>> fun_with_arg(2)
    2
    >>> fun_with_arg(2, 10)
    22
    >>> fun_with_arg(2, -5)
    -8
    """
    return a + b * 2

# may be run as below, but in most cases just a part of pytest/etc configuration
import doctest
doctest.testmod()

TestResults(failed=0, attempted=4)

# Pytest

In [ ]:
# how to run? See https://docs.pytest.org/en/latest/usage.html

In [5]:
import pytest

# do 3 different test for each argument
@pytest.mark.parametrize("some_arg", [1, 2, 3])
def test_the_same(some_arg):    
    assert some_arg > 0

# kind of stub, it's used in the in test_1_that_needs_resource_a
@pytest.fixture()
def resource_a():
    print('\nresources_a() "setup"')

def test_1_that_needs_resource_a(resource_a):
    print('test_1_that_needs_resource_a()')

# initializations and post clean methods
def resource_a_setup():
    print('resources_a_setup()')

def resource_a_teardown():
    print('resources_a_teardown()')

def setup_module(module):
    print('\nsetup_module()')
    resource_a_setup()

def teardown_module(module):
    print('\nteardown_module()')
    resource_a_teardown()


#############################################
################## one func and 2 test for it
def func(x, y):
    return x / y + 1

def test_answer():
    assert func(10, 5) == 6

def test_zero_div():
    with pytest.raises(ZeroDivisionError):
        func(10, 0)

# Unittest

In [8]:
import unittest
import math
 

class TestMath(unittest.TestCase):

    def setUp(self):
        pass
 
    def test_math_sqrt(self):
        self.assertEqual(math.sqrt(81), 9.0)
 
    def test_math_ceil_less_5(self):
        self.assertEqual(math.ceil(35.1), 36)
        
    def test_math_ceil_more_5(self):
        self.assertEqual(math.ceil(125.7), 126)
        
    def test_math_ceil_equal_5(self):
        self.assertEqual(math.ceil(0.5), 1)


# just specific args in order to run in jupyter
unittest.main(argv=['first-arg-is-ignored'], exit=False)

# in case of a separate file, just use
# unittest.main()

.....
----------------------------------------------------------------------
Ran 5 tests in 0.012s

OK


# Mock

In [6]:
import unittest
import os
import unittest.mock as mock


def fun(arg):
    return str(os.urandom(arg)) + "<<<"


def simple_urandom(length):
    return 'f' * length


class TestRandom(unittest.TestCase):

    @mock.patch('os.urandom', side_effect=simple_urandom)
    def test_urandom(self, urandom_function):
        assert fun(5) == 'fffff<<<'
        
        urandom_function.assert_called_once_with(5)

        
unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


In [19]:
# "real code", should be in a separate file 
import os
import os.path


def rm(filename):
    """
    Just a nice function, which we will test below
    """
    if os.path.isfile(filename):
        os.remove(filename)


# testing code
import unittest.mock as mock
import unittest

class RmTestCase(unittest.TestCase):
    
    @mock.patch('os.path')
    @mock.patch('os.remove')    
    def test_rm(self, mock_remove, mock_path):
        # set up the mock
        mock_path.isfile.return_value = False        
        
        rm("any path")
        
        # test that the remove call was NOT called.
        self.assertFalse(mock_remove.called,
                         "Failed to not remove the file if not present.")
        
        # make the file 'exist'
        mock_path.isfile.return_value = True
        
        rm("any path")
        
        mock_remove.assert_called_with("any path")


unittest.main(argv=['first-arg-is-ignored'], exit=False)

......
----------------------------------------------------------------------
Ran 6 tests in 0.011s

OK


# Tasks

- implement "Exercise 126: Generate All Sublists of a List" (see python workbook) and test it, include benchmarking (see https://pypi.python.org/pypi/pytest-benchmark)
- create simple flask rest api and test it (see http://flask.pocoo.org/docs/0.12/testing/)